In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import scipy.io as sio
%pylab inline

In [ ]:
# Read in worm data from SharedData .npz file
Worm = 5
worm_data = np.load('../SharedData/WormAngle'+str(Worm)+'.npz')

G = worm_data['G']
R = worm_data['R']
(neurons, time) = shape(G)
print(G.shape)

Roots of:

$\sigma_r**4*z**3 - g*r*\sigma_r**2*z**2 - r**2*\sigma_g**2*z + \sigma_g**2*\sigma_r**2*z + g**2*\sigma_r**2*z + g*r*\sigma_g**2$

Are also roots of $\frac{d(eq5)}{da} = 0$, or the values of a that $_____$ eq 5 given sigmas.

For each neuron for each timepoint

In [ ]:
# define the function for the root

def root_eq(sg, sr, g, r): 
    return ((g*r)/(3*sr**2) - (2**(1./3)*(-g**2*r**2*sr**4 - 3*r**2*sg**2*sr**4 + 3*g**2*sr**6 + 3*sg**2*sr**6))/(
        3*sr**4*(2*g**3*r**3*sr**6 + 9*g*r**3*sg**2*sr**6 - 9*g**3*r*sr**8 - 36*g*r*sg**2*sr**8 + np.sqrt(
                4*(-g**2*(r**2*sr**4 - 3*r**2*sg**2*sr**4 + 3*g**2*sr**6 + 3*sg**2*sr**6)**3 + 
                   (2*g**3*r**3*sr**6 + 9*g*r**3*sg**2*sr**6 - 9*g**3*r*sr**8 - 36*g*r*sg**2*sr**8)**2)))**(1./3))
+ (1/(3*2**(1./3)*sr**4))*((2*g**3*r**3*sr**6 + 9*g*r**3*sg**2*sr**6 - 9*g**3*r*sr**8 - 36*g*r*sg**2*sr**8 + 
                            np.sqrt(4*(-g**2*r**2*sr**4 - 3*r**2*sg**2*sr**4 + 3*g**2*sr**6 + 3*sg**2*sr**6)**3 + 
                                    (2*g**3*r**3*sr**6 + 9*g*r**3*sg**2*sr**6 - 9*g**3*r*sr**8 - 
                                     36*g*r*sg**2*sr**8)**2))**(1./3)))

In [ ]:
# Function to return a lambda with the given parameters to find the next values for sg, sr
# S is [g, r]
def create_root_lambda(g,r,a):
    return lambda s: ((g*r)/(3*s[1]**2) - (2**(1./3)*(
                -g**2*r**2*s[1]**4 - 3*r**2*s[0]**2*s[1]**4 + 3*g**2*s[1]**6 + 3*s[0]**2*s[1]**6))/
                      (3*s[1]**4*(2*g**3*r**3*s[1]**6 + 9*g*r**3*s[0]**2*s[1]**6 - 9*g**3*r*s[1]**8 - 
                                  36*g*r*s[0]**2*s[1]**8 + 
                                  np.sqrt(4*(-g**2*(r**2*s[1]**4 - 3*r**2*s[0]**2*s[1]**4 + 3*g**2*s[1]**6 + 
                                                    3*s[0]**2*s[1]**6)**3 + (2*g**3*r**3*s[1]**6 + 
                                                                             9*g*r**3*s[0]**2*s[1]**6 - 
                                                                             9*g**3*r*s[1]**8 - 36*g*r*s[0]**2*s[1]**8
                                                                            )**2)))**(1./3))
                      + (1/(3*2**(1./3)*s[1]**4))*((2*g**3*r**3*s[1]**6 + 9*g*r**3*s[0]**2*s[1]**6 - 9*g**3*r*s[1]**8
                                                    - 36*g*r*s[0]**2*s[1]**8 + np.sqrt(4*(-g**2*r**2*s[1]**4 - 3*r**2*
                                                                                          s[0]**2*s[1]**4 + 3*g**2*s[1]**6 + 3*s[0]**2*s[1]**6)**3 + 
                                    (2*g**3*r**3*s[1]**6 + 9*g*r**3*s[0]**2*s[1]**6 - 9*g**3*r*s[1]**8 - 
                                     36*g*r*s[0]**2*s[1]**8)**2))**(1./3)))-a

In [ ]:
# Find sr, sg, given a
import scipy.optimize as opt
sr = 1
sg = 1
for n in range(0,neurons):
    def fabc(s):
        f = create_root_lambda(G[n,0],R[n,0],a_r[n,0])(s)
        for t in range(1,time):
            f = f + create_root_lambda(G[n,t],R[n,t],a_r[n,t])(s)
        return f
    
    res = opt.minimize(fabc, [1,1])
    print(res.x)
# I'm not sure this is right anymore, I think we would want to minimize
# directly the equation, because the root eq should already be the closed
# form of the equation. Which is what the below does.

In [ ]:
def find_root(sr, sg):
    a_r = np.empty_like(G)

    # For each neuron
    for n in range(0,neurons):
        # For each time point
        for t in range(0,time):
            g = G[n,t]
            r = R[n,t]
            
            a_r[n,t] = root_eq(sg, sr, g, r)
    return a_r

In [ ]:
# Update sigg and sigr by maximizing eq 5:
def neg_eq_5 (g,r,a):
    return lambda s : (1/2)*(np.log(a**2*s[1]**2+s[0]**2)+(g-a*r)**2/(a**2*s[1]**2+s[0]**2))

# Find sr, sg, given a at each time
import scipy.optimize as opt
sg_comp = np.empty_like(a_r)
sr_comp = np.empty_like(a_r)
sr = 1
sg = 1
for n in range(0,neurons):
    for t in range(1,time):
        def fabc(s):
            f = neg_eq_5(G[n,t],R[n,t],a_r[n,t])(s)
            return f
    
        res = opt.minimize(fabc, [1,1])
        sg_comp[n,t] = res.x[0]
        sr_comp[n,t] = res.x[1]
        
# Find sr, sg, given a at all time for each neuron
sg_comp_all = np.empty_like(a_r[0,:])
sr_comp_all = np.empty_like(a_r[0,:])
sr = 1
sg = 1
for n in range(0,neurons):
    def fabc(s):
        f = neg_eq_5(G[n,0],R[n,0],a_r[n,0])(s)
        for t in range(1,time):
            f = f + eq_5(G[n,t],R[n,t],a_r[n,t])(s)
        return f
    
    res = opt.minimize(fabc, [1,1])
    sg_comp_all[n] = res.x[0]
    sr_comp_all[n] = res.x[1]



In [ ]:
plt.plot(sg_comp_all)
plt.plot(sr_comp_all)

In [ ]:
plt.imshow(sr_comp, aspect=10)


$ a = \frac{g}{r}$

In [ ]:
a_ratio = np.divide(G,R)


Find $\sigma_r$ and $\sigma_g$

This suggests a coordinate ascent scheme where you initialize sigr and sigg, find the a(t) at each time bin that maximizes log-likelihood, then update the sigmas (or the variances) given the a(t)'s, and repeat until convergence.  Utlimately this is an ML estimator for the T+2 length vector formed by all the a(t)'s concatenated with the sigmas.

In [ ]:
# starting sg, sr

sg = 1 
sr = 1


In [ ]:
a_r = find_root(sr=1, sg=1)

In [ ]:
f ,ax = plt.subplots(2,1, figsize=(20,2*10))
ax[0].imshow(a_ratio, aspect=10, cmap="Paired")
ax[0].set_title("g/r")
ax[1].imshow(a_r, aspect=10,cmap="Paired")
ax[1].set_title("Root a(t)")
plt.show()

Hmm, not entirely.  The root of your equation is what I'd call a_hat(t), the estimate of a(t) at each time point. So the root IS a(t) – there's no difference to minimize here.

In the second step you want to minimize equation 5 for sigg and sigr, with a_hat(t) plugged in for a(t).  So the function you're handing off to your minimizer (check this against eq 5) is:

$\sum_t    - .5  \log (a_hat(t)^2 \sigma_r + \sigma_g) - .5 *\frac{(g(t)-a_hat(t) r(t) )^2}{a_hat(t)^2 \sigma_r + \sigma_g}. $

and have it minimize that for [\vg; \vr]  (which are my shorthand for sigg^2 and and sigr^2).

Or did you already differentiate that with respect to the sigmas and set to zero?  (Does it have a nice derivative? I didn't even look but it might!)

In [ ]:
# Given an a, solve for sig r and sigg, then recover a

# Find a from g/r

# Solve for sigr and sigg
sr = -(- a^2*r^2 + 2*a*g*r - g^2 + sg)/a^2
sg = a^2*r^2 - sr*a^2 - 2*a*g*r + g^2

# Recover a

Iterate